In [ ]:
from futu import * #導入富途API模塊
import pandas as pd #導入Pandas模塊
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data = quote_ctx.get_plate_list(Market.HK, Plate.CONCEPT) #拿取所有香港概念板塊
if ret == RET_OK:
    print(data)
else:
    print('error:', data)
quote_ctx.close() # 結束後記得關閉當條連接，防止連接條數用盡

In [ ]:
data_plate_list = [] #設置一個只有板塊代碼的variable
for i in data.code:
    data_plate_list.append(i)

In [ ]:
data_plate_list

In [ ]:
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data_plate = quote_ctx.get_market_snapshot(data_plate_list) #拿取每個板塊當日的表現
if ret == RET_OK:
    print(data_plate)

else:
    print('error:', data_plate)
quote_ctx.close() # 結束後記得關閉當條連接，防止連接條數用盡

In [ ]:
data_plate = data_plate.merge(data, on='code') #把香港板塊的名字加入返每個板塊當日的表現的表

In [ ]:
data_plate.set_index('plate_name', inplace=True) #把香港板塊的名字變成索引

In [ ]:
data_plate['price_change'] = (data_plate['last_price'] / data_plate['prev_close_price']) - 1 #計算每個板塊的當日價格變動

In [ ]:
data_plate

In [ ]:
data_plate = data_plate.sort_values(by='price_change', ascending=False)[:20] #富途交易程式有相關功能 #拿取頭20表現最好的板塊
data_plate_top_20 = data_plate.code.to_list()[:20] #設置一個頭20表現最好的板塊的variable

In [ ]:
data_plate_top_20

In [ ]:
data_plate

In [ ]:
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
data_plate_top_20_table = []
for i in data_plate_top_20:
    ret, history_data, page_req_key = quote_ctx.request_history_kline(i, start='2020-10-19', end='2020-10-23')  #拿取最近5日板塊數據
    if ret == RET_OK:
        data_plate_top_20_table.append(history_data) #逐一把板塊數據加入去data_plate_top_20_table這varaible
    else:
        print('error:', data)
quote_ctx.close() # 結束後記得關閉當條連接，防止連接條數用盡

In [ ]:
data_plate_top_20_table #頭20大升幅板塊過往5日數據

In [ ]:
data_plate_top_20_table = pd.concat(data_plate_top_20_table, ignore_index= True) #變成DataFrame格式方便分析

In [ ]:
data_plate_top_20_table

In [ ]:
data_plate_top_20_table = data_plate_top_20_table.merge(data, on='code', how='left') 
#把香港板塊的名字加入返頭20大升幅板塊過往5日數據的表格

In [ ]:
data_plate_top_20_table

In [ ]:
new_table = data_plate_top_20_table.set_index(['code', 'plate_name', 'time_key']) #把香港板塊的代號﹐名字及時間變成索引

In [ ]:
new_table

In [ ]:
new_table = new_table.unstack(-1).change_rate /100 + 1 
#把時間變為Column, 香港板塊的代號﹐名字做Row﹐然後變change_rate格式轉換﹐方便使用cumprod

In [ ]:
new_table

In [ ]:
new_table.cumprod(axis=1).sort_values(by='2020-10-23 00:00:00', ascending=False) #以23/10日星期五計算，整星期升幅最多的香港板塊